In [5]:
import surp
import vice


In [6]:
import vice.yields.ccsne.settings as y_cc
import vice.yields.agb.settings as Y_agb
import vice.yields.sneia.settings as y_ia

In [7]:
surp.set_yields()

yields set to Magg et al. 2022 abundances
Yield settings
X        Z_solar    CC                             agb                            SN Ia                          

c        3.39e-03   3.69e-03 MH^2 + 1.96e-03 MH +  1.45 × cristallo11             0.00e+00                       
                    2.20e-03                                                                                     

n        1.04e-03   5.00e-04                       5.02e-04 M Z/Z0                0                              

o        7.33e-03   7.13e-03                       0                              0                              

mg       6.71e-04   6.52e-04                       0                              0                              

fe       1.37e-03   4.73e-04                       0                              7.70e-04                       





# inputs from mcmc fits

In [8]:
Y_C_AGB = {
    "cristallo11": 3.8e-4,
    "ventura13": 1.85e-4,
    "karakas16": 2.8e-4,
    "pignatari16": 5.9e-4
}

In [9]:
ZETA_C_AGB = {
    "cristallo11": -3.5e-4,
    "ventura13": -9.4e-4,
    "karakas16": -10.1e-4,
    "pignatari16": -5.7e-4
}

In [10]:
y_mg = y_cc["mg"]
noneq_factor = 1.025
noneq_zeta = 1.8
noneq_A = 1.8

Y_C_0 = 4.12 * y_mg * noneq_factor
ZETA_C_0 = 1.21 * y_mg * noneq_factor * noneq_zeta
A_C_0 = 3.07 * y_mg * noneq_factor * noneq_A

In [11]:
Y_C_0 * 0.2

0.0005506792000000001

# yield params calculator

Given the desired f_agb or alpha, what are the required y0cc and alpha_agb values

In [11]:
def f_agb_to_alpha(y0_agb, f_agb):
    y_agb = Y_C_0 * f_agb
    return y_agb / y0_agb

In [12]:
import toml

In [13]:
def f_agb_to_settings(study, y0=None, f_agb=0.2):
    if y0 is None:
        y0 = Y_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(toml.dumps(settings))

In [14]:
def f_agb_zeta_to_settings(study, *, y0_agb=None, zeta_agb=None, f_agb=0.2):
    if y0_agb is None:
        y0_agb = Y_C_AGB[study]
    if zeta_agb is None:
        zeta_agb = ZETA_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0_agb, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    
    zeta_cc = ZETA_C_0 - zeta_agb * alpha
    
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc, zeta_c_cc=zeta_cc)
    
    print(toml.dumps(settings))

In [15]:
def alpha_to_settings(study, alpha):
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(json.dumps(settings, indent=4))

In [1]:
import surp

/astro/dboyea/.local/lib/python3.11/site-packages/vice/__init__.py:141: UserWarning: Using un-released version of VICE
  warnings.warn("Using un-released version of VICE", UserWarning)


In [2]:
surp.YieldParams.from_file("../models/analytic/template/yield_params.toml")

YieldParams(yield_scale=1.0, mlr='larson1974', y_c_cc='Quadratic', y0_c_cc=0.002312, zeta_c_cc=0.0019627619263157896, kwargs_c_cc={'A': 0.0036930257999999994}, Y_c_agb='A', alpha_c_agb=1.4491557894736842, kwargs_c_agb={}, y_c_ia=0.0, y0_n_cc=0.0005, zeta_n_cc=0.0, Y_n_agb='A', kwargs_n_agb={}, eta_n_agb=0.000502, y0_n_agb=0.0, y_fe_cc=0.000473, y_fe_ia=0.00077)

In [18]:
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.8, 1]:
    print("f_agb = ", f)
    print("y0 = ", f*Y_C_0, " # AGB")
    print("y0_c_cc  = ", (1-f)* Y_C_0)
    print()

f_agb =  0
y0 =  0.0  # AGB
y0_c_cc  =  0.002753396

f_agb =  0.1
y0 =  0.00027533960000000003  # AGB
y0_c_cc  =  0.0024780564

f_agb =  0.2
y0 =  0.0005506792000000001  # AGB
y0_c_cc  =  0.0022027168000000002

f_agb =  0.3
y0 =  0.0008260187999999999  # AGB
y0_c_cc  =  0.0019273772

f_agb =  0.4
y0 =  0.0011013584000000001  # AGB
y0_c_cc  =  0.0016520375999999999

f_agb =  0.5
y0 =  0.001376698  # AGB
y0_c_cc  =  0.001376698

f_agb =  0.8
y0 =  0.0022027168000000002  # AGB
y0_c_cc  =  0.0005506791999999998

f_agb =  1
y0 =  0.002753396  # AGB
y0_c_cc  =  0.0



In [13]:
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.8, 1]:
    print("f_agb = ", f)
    f_agb_to_settings("cristallo11", f_agb=f)
    print()

f_agb =  0
alpha_c_agb = 0.0
y0_c_cc = 0.002753396


f_agb =  0.1
alpha_c_agb = 0.7245778947368421
y0_c_cc = 0.0024780564


f_agb =  0.2
alpha_c_agb = 1.4491557894736842
y0_c_cc = 0.0022027168000000002


f_agb =  0.3
alpha_c_agb = 2.173733684210526
y0_c_cc = 0.0019273772


f_agb =  0.4
alpha_c_agb = 2.8983115789473683
y0_c_cc = 0.0016520375999999999


f_agb =  0.5
alpha_c_agb = 3.6228894736842103
y0_c_cc = 0.001376698


f_agb =  0.8
alpha_c_agb = 5.796623157894737
y0_c_cc = 0.0005506791999999998


f_agb =  1
alpha_c_agb = 7.245778947368421
y0_c_cc = 0.0




In [14]:
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]:
    print("f_agb = ", f)
    f_agb_zeta_to_settings("cristallo11", f_agb=f)
    print()

f_agb =  0
alpha_c_agb = 0.0
y0_c_cc = 0.002753396
zeta_c_cc = 0.0014555574


f_agb =  0.1
alpha_c_agb = 0.7245778947368421
y0_c_cc = 0.0024780564
zeta_c_cc = 0.0017091596631578947


f_agb =  0.2
alpha_c_agb = 1.4491557894736842
y0_c_cc = 0.0022027168000000002
zeta_c_cc = 0.0019627619263157896


f_agb =  0.3
alpha_c_agb = 2.173733684210526
y0_c_cc = 0.0019273772
zeta_c_cc = 0.0022163641894736843


f_agb =  0.4
alpha_c_agb = 2.8983115789473683
y0_c_cc = 0.0016520375999999999
zeta_c_cc = 0.002469966452631579


f_agb =  0.5
alpha_c_agb = 3.6228894736842103
y0_c_cc = 0.001376698
zeta_c_cc = 0.0027235687157894737


f_agb =  1
alpha_c_agb = 7.245778947368421
y0_c_cc = 0.0
zeta_c_cc = 0.003991580031578948




In [21]:
def print_yc_tot():
    """prints out the current total c yield in an (almost nice) table"""

    print('total c yield')

    ycc = ccsne.settings["c"]
    yagb = agb.settings["c"]

    y0_cc = ycc.y0
    zeta_cc = ycc.zeta

    if isinstance(yagb, agb.interpolator):
        model = yagb.study
        alpha = yagb.prefactor
        y0_agb = alpha * Y_C_AGB[model]
        zeta_agb = alpha * ZETA_C_AGB[model]
        print("y0_agb ", Y_C_AGB[model])
        print("z0_agb ", ZETA_C_AGB[model])
    elif isinstance(yagb, C_AGB_Model):
        model = "A"
        y0_agb = yagb.y0
        zeta_agb = yagb.zeta
        print("Y agb", y0_agb)
        print("zeta agb", zeta_agb)

    print("agb_model: ", model)

    z_tot = zeta_cc + zeta_agb
    y_tot =  y0_agb + y0_cc
    f = y0_agb/y_tot

    print(f"{y_tot:0.6f} + {z_tot:0.6f} (Z-Zo)")
    print(f"f_agb = {f:0.4f}")
    print()
    print()

In [41]:
def enhance_fe_ia(factor):
    """if factor != 1, then enhances sne ia fe yield by factor but maintains same total fe yield, applied to params"""
    if factor == 1:
        return
    y_fe = y_ia["fe"] + y_cc["fe"]
    y_fe_ia = y_ia["fe"] * factor
    y_fe_cc = y_fe - y_fe_ia
    
    print(f"y_fe_ia = {y_fe_ia:0.8f}")
    print(f"y_fe_cc = {y_fe_cc:0.8f}")
    return y_fe_ia, y_fe_cc


In [42]:
enhance_fe_ia(1.2)

y_fe_ia = 0.00092400
y_fe_cc = 0.00031900


(0.0009239999999999999, 0.00031900000000000006)

### Mass factor

In [72]:
for m in [0.5, 0.8, 1, 1.5, 2]:
    surp.set_yields(y_c_cc=0, kwargs_c_agb = dict(mass_factor=m), verbose=False)
    print(m)
    print(surp.yields.calc_y(surp.Z_SUN))
    print(y_cc_0 - surp.yields.calc_y(surp.Z_SUN) + y_agb_0)

0.5
0.000420468442011002
0.0022502165963843614
0.8
0.0005204255132192604
0.002150259525176103
1
0.00046798503839536314
0.0022027
1.5
0.00047631938906597107
0.002194365649329392
2
8.347695469561248e-05
0.002587208083699751


In [71]:
y_agb_0 = 0.00046798503839536314
y_cc_0 = 0.0022027

In [72]:
for m in [0.5, 0.8, 1, 1.5, 2]:
    surp.set_yields(y_c_cc=0, kwargs_c_agb = dict(mass_factor=m), verbose=False)
    print(m)
    print(surp.yields.calc_y(surp.Z_SUN))
    print(y_cc_0 - surp.yields.calc_y(surp.Z_SUN) + y_agb_0)

0.5
0.000420468442011002
0.0022502165963843614
0.8
0.0005204255132192604
0.002150259525176103
1
0.00046798503839536314
0.0022027
1.5
0.00047631938906597107
0.002194365649329392
2
8.347695469561248e-05
0.002587208083699751


## Analytic

In [18]:
y0_agb_a = Y_C_AGB["cristallo11"]
y0_agb_a

0.00038

In [19]:
zeta_agb_a = ZETA_C_AGB["cristallo11"]
zeta_agb_a

-0.00035

In [20]:
def get_alpha(f_agb=0.2, y0_agb=y0_agb_a):
    return Y_C_0 / y0_agb * f_agb

In [21]:
def get_zeta_cc(alpha, zeta=zeta_agb_a):
    return ZETA_C_0 - alpha*zeta

In [25]:
def get_ana_settings(f, zeta=zeta_agb_a):
    alpha = get_alpha(f)
    print("alpha_agb = ", alpha)
    print("y0_c_cc = ", Y_C_0 - alpha * y0_agb_a)
    print("zeta_c_cc = ", get_zeta_cc(alpha, zeta))

In [27]:
get_ana_settings(0.2)

alpha_agb =  1.4491557894736842
y0_c_cc =  0.0022027168
zeta_c_cc =  0.0019627619263157896
